In [1]:
from dotenv import load_dotenv
load_dotenv()
from mongoengine import connect, disconnect

from settings import DevelopmentConfig, TestingConfig

In [2]:
disconnect()
connect(host=DevelopmentConfig.MONGODB_HOST)
# connect(host=TestingConfig.MONGODB_HOST)

MongoClient(host=['127.0.0.1:27016'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [3]:
from recommender.models import User, Service, Sars
from recommender.models import PytorchModule, PytorchDataset

from tests.factories.populate_database import populate_users_and_services
from recommender.engine.preprocessing import precalc_users_and_service_tensors
from recommender.models import User, Service, Sars
from recommender.engine.models.autoencoders import (
    UserAutoEncoder,
    ServiceAutoEncoder,
    USERS_AUTOENCODER,
    SERVICES_AUTOENCODER,
)
from recommender.engine.agents.pre_agent.training.common import create_embedder
from recommender.engine.utils import save_module, load_last_module
from recommender.engine.agents.rl_agent.models.history_embedder import (
    HistoryEmbedder,
    HISTORY_EMBEDDER_V1,
    HISTORY_EMBEDDER_V2,
)
from recommender.engine.agents.rl_agent.models.actor import Actor, ACTOR_V1, ACTOR_V2
from recommender.engine.agents.rl_agent.models.history_embedder import (
    HistoryEmbedder,
    HISTORY_EMBEDDER_V1,
    HISTORY_EMBEDDER_V2,
)
from recommender.engine.agents.rl_agent.preprocessing.search_data_encoder import (
    SearchDataEncoder,
)
from recommender.engine.agents.rl_agent.preprocessing.state_encoder import StateEncoder
from recommender.engine.agents.rl_agent.service_selector import ServiceSelector
from recommender.engine.agents.rl_agent.rl_agent import RLAgent

In [4]:
from recommender.engine.agents.rl_agent.training.td3_agent import TD3Agent

In [5]:
from recommender.engine.agents.rl_agent.models.critic import Critic
import numpy as np
from torch.utils.data import DataLoader
from copy import deepcopy
from recommender.engine.agents.rl_agent.reward_mapping import TRANSITION_REWARDS_CSV_PATH
import pandas as pd
import torch
from recommender.services.synthetic_dataset.dataset import generate_dataset

from recommender.engine.agents.rl_agent.training.training_agent import DDPGAgent
from tqdm.auto import tqdm, trange
import time
from recommender.services.synthetic_dataset.users import synthesize_users
from recommender.services.synthetic_dataset.env import SyntheticMP
from recommender.utils import timeit, show_times, clear_times
from recommender.engine.agents.rl_agent.training.simulation import simulate
from torch.utils.tensorboard import SummaryWriter
from definitions import LOG_DIR
from tests.factories.marketplace.category import CategoryFactory
from tests.factories.marketplace.scientific_domain import ScientificDomainFactory
from tests.factories.marketplace import *
from recommender.models import Category, ScientificDomain
import random

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from definitions import LOG_DIR
from recommender.engine.datasets.autoencoders import get_autoencoder_dataset_name, AUTOENCODERS, \
    create_autoencoder_datasets
from recommender.engine.models.autoencoders import (
    create_autoencoder_model,
    USERS_AUTOENCODER,
    SERVICES_AUTOENCODER, precalc_embedded_tensors,
)
from recommender.engine.preprocessing import USERS, SERVICES, precalc_users_and_service_tensors
from recommender.engine.training.autoencoders import (
    train_autoencoder,
    autoencoder_loss_function,
    evaluate_autoencoder,
)
from recommender.engine.utils import load_last_dataset, TRAIN, VALID, TEST, save_module, save_dataset
from recommender.models import User, Service

from recommender.engine.agents.rl_agent.training.td3_agent import TD3Agent
from collections import defaultdict

In [6]:
def gen_user_and_services(
    matching_services_numbers,
    cat_n = 10,
    sd_n = 10,
    users_n = 2000,
    user_cat_n = 5,
    user_sd_n = 5,
    services_n = 2000,
    service_cat_n = 5,
    service_sd_n = 5
):
    max_match = max(matching_services_numbers.keys())

    matching_services_number = sum(matching_services_numbers.values())
    if services_n <= matching_services_number:
        raise Exception("Too small services_n")

    not_matching_services_number = services_n - matching_services_number
    matching_services_numbers[0] = not_matching_services_number



    if cat_n <= user_cat_n or sd_n <= user_sd_n:
        raise Exception("Too small CAT_N or sd_n")

    if cat_n <= service_cat_n or sd_n <= service_sd_n:
        raise Exception("Too small CAT_N or sd_n")

    common = user_cat_n + user_sd_n
    if common < max_match:
        raise Exception("Sum of categories and scientific domains number is less than max_match")

    all_categories = CategoryFactory.create_batch(cat_n)
    user_categories = all_categories[:user_cat_n]
    not_user_categories = all_categories[user_cat_n:]

    all_scientific_domains = ScientificDomainFactory.create_batch(sd_n)
    user_scientific_domains = all_scientific_domains[:user_sd_n]
    not_user_scientific_domains = all_scientific_domains[user_sd_n:]

    user_common = user_categories + user_scientific_domains

    user = UserFactory(
        categories=user_categories,
        scientific_domains=user_scientific_domains,
        accessed_services=[],
        synthetic=True
    )
    UserFactory.create_batch(users_n-1, accessed_services=[], synthetic=True)
    total = len(matching_services_numbers.items())
    for match_n, amount in tqdm(matching_services_numbers.items(), total=total):
        for _ in trange(amount):
            common_sampled = random.sample(user_common, match_n)
            matching_c_sampled = list(filter(lambda obj: isinstance(obj, Category), common_sampled))
            not_matching_c_sampled = random.sample(not_user_categories, service_cat_n-len(matching_c_sampled))

            matching_sd_sampled = list(filter(lambda obj: isinstance(obj, ScientificDomain), common_sampled))
            not_matching_sd_sampled = random.sample(not_user_scientific_domains, service_sd_n-len(matching_sd_sampled))

            ServiceFactory(
                categories=matching_c_sampled + not_matching_c_sampled,
                scientific_domains=matching_sd_sampled + not_matching_sd_sampled,
            )

In [7]:
# for s in Service.objects:
#     s.delete()
# for u in User.objects:
#     u.delete()

In [8]:
MATCHING_SERVICES_NUMBER = {
    1:10,
    2:5,
    3:2,
    4:1,
    5:0,
}

gen_user_and_services(
    matching_services_numbers=MATCHING_SERVICES_NUMBER,
    cat_n = 10,
    sd_n = 10,
    users_n = 100,
    user_cat_n = 5,
    user_sd_n = 5,
    services_n = 100,
    service_cat_n = 5,
    service_sd_n = 5
)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

In [9]:
print(len(User.objects))
print(len(Service.objects))

100
100


In [10]:
precalc_users_and_service_tensors()

Calculating users dataframes...:   0%|          | 0/100 [00:00<?, ?it/s]

Saving users tensors...:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating services dataframes...:   0%|          | 0/100 [00:00<?, ?it/s]

/home/jan/.local/share/virtualenvs/recommender-system-r71yvAN0/lib/python3.7/site-packages/sklearn/pipeline.py:387: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return last_step.fit_transform(Xt, y, **fit_params_last_step)


Saving services tensors...:   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
device = "cpu"

In [9]:
all_datasets = {AUTOENCODERS: {}}

for collection_name in (USERS, SERVICES):
    print(f"Creating {collection_name} autoencoder datasets...")
    datasets = create_autoencoder_datasets(
        collection_name,
        train_ds_size=1.0,
        valid_ds_size=0.0,
        device=device
    )
    print(f"{collection_name} autoencoder datasets created successfully!")

    all_datasets[AUTOENCODERS][collection_name] = datasets

    print(f"Saving {collection_name} autoencoder datasets...")
    for split, dataset in datasets.items():
        save_dataset(
            dataset, name=f"{AUTOENCODERS} {collection_name} {split} dataset"
        )
    print(f"{collection_name} autoencoder datasets saved successfully!")

Creating users autoencoder datasets...
users autoencoder datasets created successfully!
Saving users autoencoder datasets...
users autoencoder datasets saved successfully!
Creating services autoencoder datasets...
services autoencoder datasets created successfully!
Saving services autoencoder datasets...
services autoencoder datasets saved successfully!


In [13]:
device = "cpu"
writer = SummaryWriter(log_dir=LOG_DIR)

In [11]:

# writer = None

UOH = len(User.objects[0].tensor)  # synthetic=True
SOH = len(Service.objects.first().tensor)

UE = 32
SE = 64

# USERS AUTOENCODER
user_autoencoder_train_ds = load_last_dataset(
    get_autoencoder_dataset_name(USERS, TRAIN)
)
# user_autoencoder_valid_ds = load_last_dataset(
#     get_autoencoder_dataset_name(USERS, VALID)
# )
# user_autoencoder_test_ds = load_last_dataset(
#     get_autoencoder_dataset_name(USERS, TEST)
# )

USER_AE_BATCH_SIZE = 128

user_autoencoder_train_ds_dl = DataLoader(
    user_autoencoder_train_ds, batch_size=USER_AE_BATCH_SIZE, shuffle=True
)
# user_autoencoder_valid_ds_dl = DataLoader(
#     user_autoencoder_valid_ds, batch_size=USER_AE_BATCH_SIZE, shuffle=True
# )
# user_autoencoder_test_ds_dl = DataLoader(
#     user_autoencoder_test_ds, batch_size=USER_AE_BATCH_SIZE, shuffle=True
# )

USER_FEATURES_DIM = len(User.objects[0].tensor)
USER_EMBEDDING_DIM = 32

user_autoencoder_model = create_autoencoder_model(
    USERS,
    features_dim=USER_FEATURES_DIM,
    embedding_dim=USER_EMBEDDING_DIM,
    writer=writer,
    train_ds_dl=user_autoencoder_train_ds_dl,
    device=device,
)

LR = 0.01
optimizer = Adam(user_autoencoder_model.parameters(), lr=LR)

EPOCHS = 200

trained_user_autoencoder_model = train_autoencoder(
    model=user_autoencoder_model,
    optimizer=optimizer,
    loss_function=autoencoder_loss_function,
    epochs=EPOCHS,
    train_ds_dl=user_autoencoder_train_ds_dl,
    # valid_ds_dl=user_autoencoder_valid_ds_dl,
    writer=writer,
    save_period=10,
    verbose=True,
    device=device,
)

loss = evaluate_autoencoder(
    trained_user_autoencoder_model,
    # user_autoencoder_test_ds_dl,
    user_autoencoder_train_ds_dl,
    autoencoder_loss_function,
    device,
)
print(f"User Autoencoder testing loss: {loss}")

save_module(trained_user_autoencoder_model, name=USERS_AUTOENCODER)



# SERVICE AUTOENCODER
service_autoencoder_train_ds = load_last_dataset(
    get_autoencoder_dataset_name(SERVICES, TRAIN)
)
# service_autoencoder_valid_ds = load_last_dataset(
#     get_autoencoder_dataset_name(SERVICES, VALID)
# )
# service_autoencoder_test_ds = load_last_dataset(
#     get_autoencoder_dataset_name(SERVICES, TEST)
# )

SERVICE_AE_BATCH_SIZE = 128

service_autoencoder_train_ds_dl = DataLoader(
    service_autoencoder_train_ds, batch_size=SERVICE_AE_BATCH_SIZE, shuffle=True
)
# service_autoencoder_valid_ds_dl = DataLoader(
#     service_autoencoder_valid_ds, batch_size=SERVICE_AE_BATCH_SIZE, shuffle=True
# )
# service_autoencoder_test_ds_dl = DataLoader(
#     service_autoencoder_test_ds, batch_size=SERVICE_AE_BATCH_SIZE, shuffle=True
# )

SERVICE_FEATURES_DIM = len(Service.objects[0].tensor)
SERVICE_EMBEDDING_DIM = 64

service_autoencoder_model = create_autoencoder_model(
    SERVICES,
    features_dim=SERVICE_FEATURES_DIM,
    embedding_dim=SERVICE_EMBEDDING_DIM,
    writer=writer,
    train_ds_dl=service_autoencoder_train_ds_dl,
    device=device,
)

LR = 0.01
optimizer = Adam(service_autoencoder_model.parameters(), lr=LR)

EPOCHS = 200

trained_service_autoencoder_model = train_autoencoder(
    model=service_autoencoder_model,
    optimizer=optimizer,
    loss_function=autoencoder_loss_function,
    epochs=EPOCHS,
    train_ds_dl=service_autoencoder_train_ds_dl,
    # valid_ds_dl=service_autoencoder_valid_ds_dl,
    writer=writer,
    save_period=10,
    verbose=True,
    device=device,
)

loss = evaluate_autoencoder(
    trained_service_autoencoder_model,
    # service_autoencoder_test_ds_dl,
    service_autoencoder_train_ds_dl,
    autoencoder_loss_function,
    device,
)
print(f"Service Autoencoder testing loss: {loss}")

save_module(trained_service_autoencoder_model, name=SERVICES_AUTOENCODER)

precalc_embedded_tensors(USERS)
precalc_embedded_tensors(SERVICES)


  0%|          | 0/1 [00:00<?, ?batch/s]

  0%|          | 0/1 [00:00<?, ?batch/s]

  0%|          | 0/1 [00:00<?, ?batch/s]

  0%|          | 0/1 [00:00<?, ?batch/s]

  0%|          | 0/1 [00:00<?, ?batch/s]

...

Total training time: 6.954431772232056
Service Autoencoder testing loss: 0.00017502307309769094


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
# for s in Service.objects[100:]:
#     s.delete()

In [14]:
# for u in User.objects[100:]:
#     u.delete()

In [11]:
print(len(Service.objects))
print(len(User.objects))

100
2000


In [13]:
def approx_service_engagement(user, service):
    common_cat_no = len(set(user.categories) & set(service.categories))
    common_sd_no = len(set(user.scientific_domains) & set(service.scientific_domains))

    x = common_cat_no + common_sd_no
    if x == 0:
        return 0
    else:
        return 1/(1+np.exp(-(x-1)))

In [14]:
user = User.objects.first()
services = list(Service.objects)
engagements = defaultdict(lambda: 0)
for service in tqdm(services, total=len(services)):
    engagement = approx_service_engagement(user, service)
    engagements[engagement] += 1
engagements

  0%|          | 0/100 [00:00<?, ?it/s]

defaultdict(<function __main__.<lambda>()>,
            {0.5: 10,
             0.7310585786300049: 5,
             0.8807970779778823: 2,
             0.9525741268224334: 1,
             0: 82})

In [18]:
# for m in PytorchModule.objects:
#     print(m.name)

In [ ]:
MAX_DEPTH = 1
env = SyntheticMP(
    N=20,
    advanced_search_data=False,
    max_depth=MAX_DEPTH,
    users_n=1
)

UE = len(User.objects.first().embedded_tensor)
SE = len(Service.objects.first().embedded_tensor)
I = len(Service.objects)

training_agent = TD3Agent(
    K=3,
    SE=SE,
    UE=UE,
    I=I,
    actor_layer_sizes=(64, 128, 256),#(64, 128, 64),
    critic_layer_sizes=(64, 128, 256), #(64, 128, 64),
    replay_buffer_max_size=1e6,
    batch_size=64,
    γ=1,
    μ_θ_α=1e-5,
    Q_Φ_α=1e-4,
    ρ=0.95,
    exploration=True,
    train_after=64,
    learning_freq=1,
    train_steps_per_update=1,
    writer=writer,
    device="cuda",
    state_encoder=None,
    service_selector=None,
    max_depth=MAX_DEPTH,
    max_steps_per_episode=env.interactions_per_user,
    act_noise=0.2,
    target_noise=0.3,
    noise_clip=0.3,
    policy_delay=2,
    act_max=1,
    act_min=-1
)

simulate(
    env,
    training_agent,
    episodes=10000,
    render=False,
    max_episode_steps=None,
)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
show_times()
# clear_times()

[act] Mean execution time: 0.45726110935211184, records number: 20
[step] Mean execution time: 0.18867028951644899, records number: 20
[observe] Mean execution time: 0.8520666674563759, records number: 19


{'act': [0.7820796966552734,
  0.4354872703552246,
  0.41303467750549316,
  0.41414546966552734,
  0.4141998291015625,
  0.4132344722747803,
  0.41949892044067383,
  0.41841959953308105,
  0.7876832485198975,
  0.406292200088501,
  0.4190216064453125,
  0.4116792678833008,
  0.4106581211090088,
  0.41650915145874023,
  0.41257190704345703,
  0.4294857978820801,
  0.4265906810760498,
  0.43399906158447266,
  0.41211414337158203,
  0.4685170650482178],
 'step': [0.23380661010742188,
  0.17930269241333008,
  0.1784217357635498,
  0.18072009086608887,
  0.1789531707763672,
  0.17786049842834473,
  0.17998123168945312,
  0.18220233917236328,
  0.256389856338501,
  0.18092083930969238,
  0.17880749702453613,
  0.17922329902648926,
  0.1805739402770996,
  0.18253231048583984,
  0.17981839179992676,
  0.18604397773742676,
  0.1967332363128662,
  0.17963576316833496,
  0.17916297912597656,
  0.2023153305053711],
 'observe': [0.7956383228302002,
  0.8069610595703125,
  0.8001120090484619,
  0.81

In [ ]:
len(User.objects)

In [15]:
users_sd_ids = list(map(lambda sd: sd.id, user.scientific_domains))
users_c_ids = list(map(lambda c: c.id, user.categories))

In [17]:
print(f"User has {len(users_sd_ids)} scientific domains with following ids:")
print(users_sd_ids)
print(f"... and it has {len(users_c_ids)} categories with following ids:")
print(users_c_ids)

User has 10 scientific domains with following ids:
[26, 62, 71, 20, 3, 73, 57, 64, 7, 15]
... and it has 12 categories with following ids:
[1, 10, 24, 26, 28, 32, 46, 56, 93, 136, 197, 199]


In [108]:
{x: 1/(1+np.exp(-(x-1))) for x in range(1,7)}

{1: 0.5,
 2: 0.7310585786300049,
 3: 0.8807970779778823,
 4: 0.9525741268224334,
 5: 0.9820137900379085,
 6: 0.9933071490757153}

In [74]:
engagements = defaultdict(lambda: 0)
for s in Service.objects:
    eng = approx_service_engagement(user, s)
    engagements[eng] += 1

In [86]:
from pprint import pprint

In [92]:
dict(engagements)

{0: 139, 0.5: 177, 0.7310585786300049: 9, 0.8807970779778823: 2}

In [73]:
from collections import defaultdict

In [10]:
d = defaultdict(set)
for s in Service.objects:
    for c in s.categories:
        d[s.id].add(c)

In [19]:
amounts = {k: len(v) for k, v in d.items()}

In [29]:
max(amounts.values())

7

In [23]:
[id for id, amount in amounts.items() if amount > 1]

[220,
 245,
 487,
 4,
 7,
 408,
 10,
 8,
 490,
 48,
 428,
 78,
 3,
 1,
 412,
 15,
 482,
 58,
 18,
 6,
 9,
 11,
 438,
 81,
 2,
 430,
 5,
 485,
 392,
 0]

In [28]:
Service.objects(id=487).first().categories

[<Category: Category object>,
 <Category: Category object>,
 <Category: Category object>,
 <Category: Category object>,
 <Category: Category object>]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 4,
 2,
 1,
 5,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,


In [4]:
# precalc_users_and_service_tensors()

In [5]:
writer = SummaryWriter(log_dir=LOG_DIR)

In [6]:
%%time
MAX_DEPTH = 10
env = SyntheticMP(
    N=20,
    advanced_search_data=False,
    max_depth=MAX_DEPTH
)

CPU times: user 426 ms, sys: 10.4 ms, total: 437 ms
Wall time: 442 ms


In [7]:
UE = len(User.objects.first().embedded_tensor)
SE = len(Service.objects.first().embedded_tensor)
I = len(Service.objects)

In [8]:
training_agent = DDPGAgent(
    K=3,
    SE=SE,
    UE=UE,
    I=I,
    actor_layer_sizes=(256, 512, 256),#(64, 128, 64),
    critic_layer_sizes=(256, 512, 256), #(64, 128, 64),
    replay_buffer_max_size=1e6,
    batch_size=32,
    γ=0.995,
    μ_θ_α=1e-4,
    Q_Φ_α=1e-3,
    ρ=0.95,
    exploration=True,
    noise_sigma=0.1,
    train_after=20, # TODO: because batchnorm needs more than one value - it can still be not enough due to invalid sarses problem
    learning_freq=1,
    train_steps_per_update=1,
#     writer=writer,
    writer=None,
    device="cuda",
    state_encoder=None,
    service_selector=None,
    max_depth=MAX_DEPTH,
    max_steps_per_episode=env.interactions_per_user
)

In [9]:
import pickle
import os

In [10]:
temp_dir = os.path.join("/home/jan/cyfronet/fid_team/deployment_test/recommender-system/", "temp_dir")
temp_dir

'/home/jan/cyfronet/fid_team/deployment_test/recommender-system/temp_dir'

In [11]:
training_agent._last_S = None

In [15]:
training_agent.save(temp_dir)

Agent saved successfully! (agent.writer object can't be saved so this field has been set to `None` in saved agent, but hasn't been changed in current agent object). _last_S couldn't be saved also.


In [13]:
with open(temp_dir, "wb") as file:
    pickle.dump(training_agent, file)

In [14]:
simulate(
    env,
    training_agent,
    episodes=1,
    render=False,
    max_episode_steps=1,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]